# Quick Start

## Launch a server

This code uses `subprocess.Popen` to start an SGLang server process, equivalent to executing 

```bash
python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
--port 30000 --host 0.0.0.0 --log-level warning
```
in your command line and wait for the server to be ready.

In [1]:
# import subprocess
# import time
# import requests
# import os

# server_process = subprocess.Popen(
#     [
#         "python",
#         "-m",
#         "sglang.launch_server",
#         "--model-path",
#         "meta-llama/Meta-Llama-3.1-8B-Instruct",
#         "--port",
#         "30000",
#         "--host",
#         "0.0.0.0",
#         "--log-level",
#         "error",
#     ],
#     text=True,
#     stdout=subprocess.DEVNULL,
#     stderr=subprocess.DEVNULL,
# )

# while True:
#     try:
#         response = requests.get(
#             "http://localhost:30000/v1/models",
#             headers={"Authorization": "Bearer None"},
#         )
#         if response.status_code == 200:
#             break
#     except requests.exceptions.RequestException:
#         time.sleep(1)

# print("Server is ready. Proceeding with the next steps.")

## Send a Request

Once the server is running, you can send test requests using curl.

In [2]:
# !curl http://localhost:30000/v1/chat/completions \
#   -H "Content-Type: application/json" \
#   -H "Authorization: Bearer None" \
#   -d '{"model": "meta-llama/Meta-Llama-3.1-8B-Instruct", "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "What is a LLM?"}]}'

## Using OpenAI Compatible API

SGLang supports OpenAI-compatible APIs. Here are Python examples:

In [3]:
# import openai

# # Always assign an api_key, even if not specified during server initialization.
# # Setting an API key during server initialization is strongly recommended.

# client = openai.Client(
#     base_url="http://127.0.0.1:30000/v1", api_key="None"
# )

# # Chat completion example

# response = client.chat.completions.create(
#     model="meta-llama/Meta-Llama-3.1-8B-Instruct",
#     messages=[
#         {"role": "system", "content": "You are a helpful AI assistant"},
#         {"role": "user", "content": "List 3 countries and their capitals."},
#     ],
#     temperature=0,
#     max_tokens=64,
# )
# print(response)

In [4]:
# import signal  # Add this at the top with other imports
# import gc  # Add this for garbage collection
# import torch

# def terminate_process(process):
#     try:
#         # Gracefully terminate the process
#         process.terminate()
#         try:
#             process.wait(timeout=5)
#         except subprocess.TimeoutExpired:
#             if os.name != 'nt':  # Unix-like systems
#                 try:
#                     pgid = os.getpgid(process.pid)
#                     os.killpg(pgid, signal.SIGTERM)
#                     time.sleep(1)
#                     if process.poll() is None:
#                         os.killpg(pgid, signal.SIGKILL)
#                 except ProcessLookupError:
#                     pass
#             else:  # Windows
#                 process.kill()
#             process.wait()
#     except Exception as e:
#         print(f"Warning: {e}")
#     finally:
#         # Force cleanup
#         gc.collect()
#         if torch.cuda.is_available():  # Add this import if using PyTorch
#             torch.cuda.empty_cache()
#             torch.cuda.ipc_collect()

# # When terminating the first server
# terminate_process(server_process)
# time.sleep(2)  # Give some time for cleanup